In [1]:
import chainer
import chainer.links as L
import chainer.functions as F
import chainer.optimizers as opt
from chainer.dataset import concat_examples
from chainer.backends import cuda

import numpy as np
from functools import partial

#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm

import models.VGG

# DataLoader(CIFAR10)

In [2]:
batch_size = 100
def transform(in_data, std, mean):
    img, label = in_data
    img -= np.asarray(mean)[:, np.newaxis, np.newaxis]
    img /= np.asarray(std)[:, np.newaxis, np.newaxis]
    return img, label
trainset, testset = chainer.datasets.cifar.get_cifar10(withlabel=True, ndim=3, scale=1.0)

trainset_trans = chainer.datasets.TransformDataset(trainset, partial(transform, std=[0.5, 0.5, 0.5], mean=[0.5, 0.5, 0.5]))
train_loader = chainer.iterators.MultiprocessIterator(trainset_trans, batch_size=batch_size, shuffle = True, n_processes=2)

testset_trans = chainer.datasets.TransformDataset(testset, partial(transform, std=[0.5, 0.5, 0.5], mean=[0.5, 0.5, 0.5]))
test_loader = chainer.iterators.MultiprocessIterator(testset_trans, batch_size=batch_size, shuffle = False, n_processes=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model

In [3]:
model = L.VGG16Layers()
model.fc6 = L.Linear(512, 4096)
model.fc8 = L.Linear(4096, len(classes))
#model.available_layers

##model = L.Classifier(models.VGG.VGG(len(classes)))
#model = models.VGG.VGG(len(classes))

gpu_id = 0  # Set to -1 if you use CPU
if gpu_id >= 0:
    model.to_gpu(gpu_id)

# Optimizer

In [4]:
optimizer = opt.Adam(alpha=0.001, beta1=0.9, beta2=0.999, eps=1e-08, eta=1.0, weight_decay_rate=0, amsgrad=False) 
optimizer.setup(model)

# Train 1 epoch function

In [5]:
def train_loop(model, loader, optimizer, num_steps,gpu_id):
    with chainer.using_config('train', True):
        bar = tqdm(total=num_steps, leave=False)
        total_loss, total_acc, total_num = 0, 0, 0
        while True:
            feed = loader.next()
            # Prepare data
            inputs, labels = concat_examples(feed, gpu_id)
            # Foward
            outputs = model.forward(inputs, layers=['fc8'])['fc8']
            #outputs = model(inputs)
            # Calcurate Loss
            loss = F.softmax_cross_entropy(outputs, labels)
            # initialize gradient
            model.cleargrads()
            # Backward
            loss.backward()
            # Update Params
            optimizer.update()
            # Update bar
            ## Accuracy
            accuracy = F.accuracy(outputs, labels)
            accuracy.to_cpu()
            ## Calcurate Score
            total_loss += chainer.cuda.to_cpu(loss.data) * len(labels)
            total_acc += accuracy.data
            total_num += len(labels)

            #print(total_loss / total_num, total_acc / total_num * 100)
            bar.set_description("Loss: {:.4f}, Accuracy: {:.2f}".format(
                total_loss / total_num, total_acc / total_num * 100), refresh=True)
            bar.update()
            
            if loader.is_new_epoch:
                bar.close()
                break
                
    return total_loss / total_num, total_acc / total_num * 100

# Validation function

In [6]:
def valid_loop(model, loader, optimizer, num_steps,gpu_id):
    with chainer.using_config('train', False):
        bar = tqdm(total=num_steps, leave=False)
        total_loss, total_acc, total_num = 0, 0, 0
        while True:
            feed = loader.next()
            # Prepare data
            inputs, labels = concat_examples(feed, gpu_id)
            # Foward
            outputs = model.forward(inputs, layers=['fc8'])['fc8']
            #outputs = model(inputs)
            # Calcurate Loss
            loss = F.softmax_cross_entropy(outputs, labels)
            # Update bar
            ## Accuracy
            accuracy = F.accuracy(outputs, labels)
            accuracy.to_cpu()
            ## Calcurate Score
            total_loss += chainer.cuda.to_cpu(loss.data) * len(labels)
            total_acc += accuracy.data
            total_num += len(labels)

            #print(total_loss / total_num, total_acc / total_num * 100)
            bar.set_description("Loss: {:.4f}, Accuracy: {:.2f}".format(
                total_loss / total_num, total_acc / total_num * 100), refresh=True)
            bar.update()
            
            if loader.is_new_epoch:
                bar.close()
                break
                
    return total_loss / total_num, total_acc / total_num * 100

In [7]:
max_epochs = 5
for e in range(max_epochs):
    train_loss, train_acc = train_loop(model, train_loader, optimizer, len(trainset_trans)/batch_size, gpu_id)
    test_loss, test_acc = valid_loop(model, test_loader, optimizer, len(testset_trans)/batch_size, gpu_id)
    print('Epoch: {}, Train Loss: {:.4f}, Train Accuracy: {:.2f}, Test Loss: {:.4f}, Test Accuracy: {:.2f}'.format(
    e + 1, train_loss, train_acc, test_loss, test_acc))

Epoch: 1, Train Loss: 2.0281, Train Accuracy: 0.21, Test Loss: 1.7474, Test Accuracy: 0.27


Epoch: 2, Train Loss: 1.6059, Train Accuracy: 0.35, Test Loss: 1.3280, Test Accuracy: 0.49


Epoch: 3, Train Loss: 1.3217, Train Accuracy: 0.50, Test Loss: 1.3075, Test Accuracy: 0.53


Epoch: 4, Train Loss: 1.1075, Train Accuracy: 0.61, Test Loss: 1.0214, Test Accuracy: 0.64


Epoch: 5, Train Loss: 0.9258, Train Accuracy: 0.69, Test Loss: 0.9036, Test Accuracy: 0.69


In [8]:
chainer.serializers.save_npz('params/model.npz', model)